In [ ]:
import pandas as pd
import numpy as np

import urllib.request, urllib.parse, json
import pickle

# Import crop data

In [ ]:
crop_data = pd.read_csv("/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Geo_GW_APY_Data_Step2.csv")

In [ ]:
crop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214471 entries, 0 to 214470
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   State                     214471 non-null  object 
 1   District                  214471 non-null  object 
 2   Lat                       214471 non-null  float64
 3   Long                      214471 non-null  float64
 4   GW_depth_Minimum          214471 non-null  float64
 5   GW_depth_Maximum          214471 non-null  float64
 6   Wells_depth_0_to_2        214471 non-null  int64  
 7   Wells_depth_2_to_5        214471 non-null  int64  
 8   Wells_depth_5_to_10       214471 non-null  int64  
 9   Wells_depth_10_to_20      214471 non-null  float64
 10  Wells_depth_20_to_40      214471 non-null  float64
 11  Wells_depth_40_and_above  214471 non-null  float64
 12  Crop_Year                 214471 non-null  int64  
 13  Season                    214471 non-null  o

In [ ]:
crop_data

,State,District,Lat,Long,GW_depth_Minimum,GW_depth_Maximum,Wells_depth_0_to_2,Wells_depth_2_to_5,Wells_depth_5_to_10,Wells_depth_10_to_20,Wells_depth_20_to_40,Wells_depth_40_and_above,Crop_Year,Season,Crop,Area,Production
0,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Arecanut,3100.0,5200.0
1,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Other Kharif pulses,668.0,448.0
2,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Rice,10779.0,31863.0
3,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Banana,1531.0,12073.0
4,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Cashewnut,80.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214466,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Summer,Rice,195960.0,580442.0
214467,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Summer,Sesamum,74080.0,58514.0
214468,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Whole Year,Sugarcane,4390.0,789614.0
214469,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Winter,Rice,496134.0,1348329.0


In [ ]:
df_crop = crop_data.copy()

In [ ]:
df_crop.columns

Index(['State', 'District', 'Lat', 'Long', 'GW_depth_Minimum',
       'GW_depth_Maximum', 'Wells_depth_0_to_2', 'Wells_depth_2_to_5',
       'Wells_depth_5_to_10', 'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Crop_Year', 'Season', 'Crop', 'Area',
       'Production'],
      dtype='object')

In [ ]:
df_crop.columns = ['State', 'District', 'Lat', 'Long', 'GW_depth_Minimum',
       'GW_depth_Maximum', 'Wells_depth_0_to_2', 'Wells_depth_2_to_5',
       'Wells_depth_5_to_10', 'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Year', 'Season', 'Crop', 'Area',
       'Production']

In [ ]:
df_crop['Season'] = df_crop['Season'].apply(lambda s: s.strip())
seasons = df_crop['Season'].unique()
seasons

array(['Kharif', 'Whole Year', 'Autumn', 'Rabi', 'Summer', 'Winter'],
      dtype=object)

In [ ]:
df_crop['Lat'] = df_crop['Lat'].apply(lambda lat: np.round(lat, 5))
df_crop['Long'] = df_crop['Long'].apply(lambda lon: np.round(lon, 5))

In [ ]:
# Mapping of crop seasons to its months
season_months = {
    'Whole Year': list(range(1, 13)),
    'Autumn': list(range(9, 12)),
    'Kharif': list(range(6, 11)),
    'Rabi': [11, 12, 1, 2, 3],
    'Summer': [3, 4, 5, 6],
    'Winter': [12, 1]
}


# Get Weather data from CSV or server



In [ ]:
lat_long_values = df_crop[['Lat', 'Long']].values

In [ ]:
lat_long_values = np.unique(lat_long_values, axis=0)
lat_long_values[:10]

array([[ 8.08783, 77.53841],
       [ 8.381  , 77.6132 ],
       [ 8.56224, 76.92684],
       [ 8.80545, 77.67275],
       [ 8.87875, 76.76535],
       [ 9.26485, 76.78995],
       [ 9.34366, 78.86305],
       [ 9.46214, 76.28424],
       [ 9.63726, 76.60583],
       [ 9.8139 , 77.00703]])

In [ ]:
# NASA API Dictionary
# ALLSKY_SFC_SW_DWN - Average amount of solar radiation incident on a horizontal surface at the surface of the earth under all-sky conditions (MJ/m^2/day)
# PRECTOT - Precipitation (mm)
# PS - Surface Pressure (kPa)
# RH2M - Relative Humidity at 2m (g/kg)
# T2MDEW - Dew/Frost Point at 2m (C)
# T2M_MAX - Max. Temperature at 2m (C)
# T2M_MIN - Min. Temperature at 2m (C)
# WS2M - Wind Speed at 2m (m/s)

weather_data = {}

try:
  in_file = open('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Weather/weather_data.pkl', 'rb')
  weather_data = pickle.load(in_file)                     
  in_file.close()
  print(f"Weather data found - {len(weather_data)}")
except:
  weather_data = {}
  for (lat, lon) in lat_long_values:
    req_url = f"https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=PRECTOT,RH2M,PS,T2M_MAX,T2M_MIN,T2MDEW,WS2M,ALLSKY_SFC_SW_DWN&startDate=1996&endDate=2015&userCommunity=AG&tempAverage=INTERANNUAL&outputList=ASCII&lat={lat}&lon={lon}"
    with urllib.request.urlopen(req_url) as url:
      data = json.loads(url.read().decode())
      if lat not in weather_data.keys():
        weather_data[lat] = {}
      if lon not in weather_data[lat].keys():
        weather_data[lat][lon] = {}
      weather_data[lat][lon] = data['features'][0]['properties']['parameter']

  # write weather data to pkl file
  out_file = open('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Weather/weather_data.pkl', 'ab')
  pickle.dump(weather_data, out_file)
  out_file.close()

In [ ]:
# Find averages based on season
geolocation_weather_by_year = []

for row in df_crop[['Lat', 'Long', 'Season', 'Year']].values:
  lat, lon, season, year = row
  months = season_months[season]
  geo_weather = weather_data[lat][lon]
  parameters = list(geo_weather.keys())
  temp_data = {}
  for param in parameters:
    required_year = year
    if months[-1] - months[0] < 0:
      required_year -= 1
    last_month = 0
    if param not in temp_data:
      temp_data[param] = []
    for month in months:
      if month < last_month:
        required_year += 1
      required_key = str(str(required_year) + str('%02d' % month))
      last_month = month
      temp_data[param].append(geo_weather[param][required_key])
  geolocation_weather_by_year.append({
    "Lat": lat,
    "Long": lon,
    "Season": season,
    "Year": year,
    "Solar_Radiation": np.mean(temp_data['ALLSKY_SFC_SW_DWN']),
    "Precipitation": np.mean(temp_data['PRECTOT']),
    "Surface_Pressure": np.mean(temp_data['PS']),
    "Humidity": np.round(np.mean(temp_data['RH2M']), 3),
    "Dew_Frost_Point": np.round(np.mean(temp_data['T2MDEW']), 3),
    "Temp_Max": np.round(np.mean(temp_data['T2M_MAX']), 3),
    "Temp_Min": np.round(np.mean(temp_data['T2M_MIN']), 3),
    "Wind_Speed": np.round(np.mean(temp_data['WS2M']), 3)
  })

df_weather = pd.DataFrame(geolocation_weather_by_year)

In [ ]:
df_weather = df_weather.sort_values('Year').reset_index(drop=True)
df_weather

,Lat,Long,Season,Year,Solar_Radiation,Precipitation,Surface_Pressure,Humidity,Dew_Frost_Point,Temp_Max,Temp_Min,Wind_Speed
0,25.37851,77.82390,Kharif,1997,17.848,144.666,96.010,67.614,20.924,33.012,23.374,2.576
1,26.24172,74.79064,Kharif,1997,18.670,102.204,95.984,60.434,19.752,34.396,23.486,2.560
2,26.24172,74.79064,Kharif,1997,18.670,102.204,95.984,60.434,19.752,34.396,23.486,2.560
3,26.24172,74.79064,Kharif,1997,18.670,102.204,95.984,60.434,19.752,34.396,23.486,2.560
4,26.24172,74.79064,Kharif,1997,18.670,102.204,95.984,60.434,19.752,34.396,23.486,2.560
...,...,...,...,...,...,...,...,...,...,...,...,...
214466,21.09947,84.95384,Winter,2015,15.000,17.810,98.705,60.045,9.615,25.275,11.185,1.180
214467,21.09947,84.95384,Winter,2015,15.000,17.810,98.705,60.045,9.615,25.275,11.185,1.180
214468,21.09947,84.95384,Winter,2015,15.000,17.810,98.705,60.045,9.615,25.275,11.185,1.180
214469,21.09947,84.95384,Winter,2015,15.000,17.810,98.705,60.045,9.615,25.275,11.185,1.180


In [ ]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214471 entries, 0 to 214470
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Lat               214471 non-null  float64
 1   Long              214471 non-null  float64
 2   Season            214471 non-null  object 
 3   Year              214471 non-null  int64  
 4   Solar_Radiation   214471 non-null  float64
 5   Precipitation     214471 non-null  float64
 6   Surface_Pressure  214471 non-null  float64
 7   Humidity          214471 non-null  float64
 8   Dew_Frost_Point   214471 non-null  float64
 9   Temp_Max          214471 non-null  float64
 10  Temp_Min          214471 non-null  float64
 11  Wind_Speed        214471 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 19.6+ MB


In [ ]:
df_weather = df_weather.drop_duplicates()
# df_weather.to_csv('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Weather/Geolocation_wise_Weather.csv', index=False)

In [ ]:
print(df_crop.shape, df_weather.shape)

(214471, 17) (27330, 12)


In [ ]:
df_merged = df_crop.merge(df_weather, on=['Lat', 'Long', 'Season', 'Year'], how='inner').reset_index(drop=True)

In [ ]:
df_merged.shape

(214471, 25)

In [ ]:
df_merged.isna().sum()

State                       0
District                    0
Lat                         0
Long                        0
GW_depth_Minimum            0
GW_depth_Maximum            0
Wells_depth_0_to_2          0
Wells_depth_2_to_5          0
Wells_depth_5_to_10         0
Wells_depth_10_to_20        0
Wells_depth_20_to_40        0
Wells_depth_40_and_above    0
Year                        0
Season                      0
Crop                        0
Area                        0
Production                  0
Solar_Radiation             0
Precipitation               0
Surface_Pressure            0
Humidity                    0
Dew_Frost_Point             0
Temp_Max                    0
Temp_Min                    0
Wind_Speed                  0
dtype: int64

In [ ]:
df_merged

,State,District,Lat,Long,GW_depth_Minimum,GW_depth_Maximum,Wells_depth_0_to_2,Wells_depth_2_to_5,Wells_depth_5_to_10,Wells_depth_10_to_20,Wells_depth_20_to_40,Wells_depth_40_and_above,Year,Season,Crop,Area,Production,Solar_Radiation,Precipitation,Surface_Pressure,Humidity,Dew_Frost_Point,Temp_Max,Temp_Min,Wind_Speed
0,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Arecanut,3100.0,5200.0,16.4440,303.200000,100.636000,84.786,24.768,27.886,27.200,5.766
1,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Other Kharif pulses,668.0,448.0,16.4440,303.200000,100.636000,84.786,24.768,27.886,27.200,5.766
2,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Rice,10779.0,31863.0,16.4440,303.200000,100.636000,84.786,24.768,27.886,27.200,5.766
3,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Banana,1531.0,12073.0,18.6575,201.552500,100.761667,80.225,23.741,28.057,26.927,4.752
4,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Cashewnut,80.0,54.0,18.6575,201.552500,100.761667,80.225,23.741,28.057,26.927,4.752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214466,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Summer,Rice,195960.0,580442.0,19.9075,108.287500,100.000000,58.630,20.400,36.832,24.740,2.388
214467,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Summer,Sesamum,74080.0,58514.0,19.9075,108.287500,100.000000,58.630,20.400,36.832,24.740,2.388
214468,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Whole Year,Sugarcane,4390.0,789614.0,16.7950,125.891667,100.258333,70.648,19.262,31.058,20.652,2.059
214469,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Winter,Rice,496134.0,1348329.0,14.2150,0.275000,101.040000,73.865,12.620,24.160,11.495,1.570


In [ ]:
df_merged.columns

Index(['State', 'District', 'Lat', 'Long', 'GW_depth_Minimum',
       'GW_depth_Maximum', 'Wells_depth_0_to_2', 'Wells_depth_2_to_5',
       'Wells_depth_5_to_10', 'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Year', 'Season', 'Crop', 'Area',
       'Production', 'Solar_Radiation', 'Precipitation', 'Surface_Pressure',
       'Humidity', 'Dew_Frost_Point', 'Temp_Max', 'Temp_Min', 'Wind_Speed'],
      dtype='object')

In [ ]:
rearranged_columns = ['State', 'District', 'Lat', 'Long', 'GW_depth_Minimum',
       'GW_depth_Maximum', 'Wells_depth_0_to_2', 'Wells_depth_2_to_5',
       'Wells_depth_5_to_10', 'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Precipitation', 'Solar_Radiation', 'Surface_Pressure', 'Humidity',
       'Temp_Max', 'Temp_Min', 'Dew_Frost_Point', 'Wind_Speed', 'Year', 'Season', 'Crop', 'Area', 'Production']
df_merged = df_merged[rearranged_columns]

In [ ]:
df_merged.columns = ['State', 'District', 'Lat', 'Long', 'GW_depth_Min',
       'GW_depth_Max', 'Wells_depth_0_to_2', 'Wells_depth_2_to_5',
       'Wells_depth_5_to_10', 'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Precipitation', 'Solar_Radiation', 'Surface_Pressure', 'Humidity',
       'Temp_Max', 'Temp_Min', 'Dew_Frost_Point', 'Wind_Speed', 'Year', 'Season', 'Crop', 'Area', 'Production']

In [ ]:
df_merged.to_csv('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Merged_Weather_Data_Step3.csv', index=False)